In [1]:
import os
import luigi
import io
import pandas as pd
import numpy as np
# from emu.pipeline.download import Channel, Raw
# from src.pipeline.utils import file_ids_by_channel
# from src.luigi.box import BoxTarget
# from src.utils import get_file_manifest

# Patient Manifest
`/EMU/_patient_manifest.csv` maps every patient to their corresponding Data folders

It can be loaded quickly using the `load_patients` helper function

In [2]:
from emu.utils import load_patients
patient_manifest = load_patients()
patient_manifest

,patient_id,study,folder_id,start,type
0,0,test,94102978809,'2019-07-30_11-04-51',electrophysiology
1,1,pdil,105837046151,NaN,study_root
2,1,pdil,105997042269,NaN,behavioral
3,1,pdil,105835938489,NaN,behavioral
4,1,doubt,105837046151,NaN,study_root


The patient_id, study, and folder_id's are likely self explanatory, type refers to what kind of data is stored at the corresponding folder_id.

> A single patient in the same experiment might have data spread across multiple days

# File Manifest

You can load all relevant files in subdirectories of a given folder using `get_file_manifest`

In [3]:
from emu.utils import get_file_manifest
from emu.luigi.box import BoxClient
folder_id = 105997042269
client = BoxClient()
box_folder = client.folder(folder_id)
print(box_folder.get())

<Box Folder - 105997042269 (PT01_postOpDay4_PDil)>


In [6]:
# Keep only folders referencing behavioral data
behavioral_folders = patient_manifest[patient_manifest.type.isin(['behavioral'])]

all_files = []
for fid in behavioral_folders.folder_id.values:
    bf = client.folder(fid)
    all_files.extend(get_file_manifest(bf))
    
all_files = pd.DataFrame.from_records(all_files)
all_files

PT01_postOpDay2_PDil: 100%|██████████| 5/5 [00:00<00:00, 14.33it/s]


,filename,id,type
0,blockNum_3_computerTT_defect_taskoutput.mat,629615908637,Matfile
1,blockNum_4_humanTT_defect_taskoutput.mat,629620274728,Matfile
2,blockNum_5_humanTT_coop_taskoutput.mat,629611713944,Matfile
3,blockNum_6_computerTT_coop_taskoutput.mat,629611688205,Matfile
4,Survey_postBlock3.jpeg,630722640790,None
5,Survey_postBlock4.jpeg,630717432226,None
6,Survey_postBlock5.jpeg,630716805900,None
7,Survey_postBlock6.jpeg,630719096807,None
8,blockNum_1_computerTT_coop_taskoutput.mat,628721484958,Matfile
9,blockNum_2_humanTT_coop_taskoutput.mat,628728186668,Matfile


In [5]:
# Make sure we have the FileManifest for patient 1
pre_tasks = [FileManifest(patient_id=1)]
luigi.build(pre_tasks, local_scheduler=True)

fm_output = pre_tasks[0].output()
with fm_output.open('r') as infile:
    files = pd.read_csv(infile,dtype={'filename':str,'type':str, 'id':np.int,'path':str})

NameError: name 'FileManifest' is not defined

In [ ]:
files

In [ ]:
ch_files = file_ids_by_channel(files,channel_ids=[2])
target_dir = os.path.join(root_dir,'raw','pt{}'.format(1))

In [ ]:
fetch_channels = []
for fid,fn in zip(ch_files.id.values,ch_files.filename.values):
    fetch_channels.append(Raw(file_id=fid,save_to=target_dir,file_name=fn))

In [ ]:
for f in fetch_channels:
    print(f)